In [4]:
import evoVAE.utils.seq_tools as st
from evoVAE.utils.datasets import MSA_Dataset
import evoVAE.utils.statistics as stats
import pandas as pd
import torch
import numpy as np
import concurrent.futures
import threading

from scipy import stats
from Bio import Align, AlignIO


pd.set_option("display.max_rows", None)

In [59]:

def trim_aln(aln : Align.MultipleSeqAlignment, cols):
    """ Trim indices in cols from a multiple sequence alignment. """

    # Check that all col indices are in range of sequence length
    if not isinstance(cols, list): cols = [cols]
    cols = list(set(cols))  # Remove duplicate column indices
    for col in cols:
        if not 0 <= col < aln.get_alignment_length():
            raise RuntimeError(f"Invalid column index: {col}")

    # Trim from highest to lowest index
    cols.sort()
    for col in cols[::-1]:
        if col == aln.get_alignment_length() - 1:  # Trimming last col of current aln
            aln = aln[:, :-1]
        elif col == 0:    # Trimming first column
            aln = aln[:, 1:]
        else:   # Trimming non-end column and need to combine two sub-alns
            aln = aln[:, :col] + aln[:, col+1:]

    return aln


def smart_trim(in_file, out_file=None, format='fasta', base_threshold=0.05, p_threshold=0.05, max_threads=10):
    """ Trim columns informed by column occupancy and pairwise sequence similarity of sequences with characters in
     low-occupancy positions. Trimming in this manner is likely to remove erroneously aligned columns without a large
     loss of phylogenetic signal, and reduce complexity for indel resolution in ASR. Note that if ASR is being performed
     using the resulting alignment, an assessment should be made as to whether trimmed columns are likely to be present
     in any targetted node. """

    full_aln = AlignIO.read(in_file, format)

    pw_coverage_full = {}

    # Multi-thread pairwise coverage calculations
    pw_coverage_executor = concurrent.futures.ThreadPoolExecutor(max_workers=max_threads)
    pw_coverage_lock = threading.Lock()

    # Method for pairwise coverage calculation of sequence j with all sequences from j+1 to len(aln)
    def pw_coverage(j):
        # Calculate pairwise alignment coverages (as aligned proportion of shorter sequence)
        for k in range(j + 1, len(full_aln)):
            seqs = {full_aln[j].name: full_aln[j], full_aln[k].name: full_aln[k]}
            # Determine shortest ungapped seq
            shorter = min(seqs.keys(), key=lambda seq: len(str(seqs[seq].seq).replace('-', '')))
            longer = [key for key in seqs.keys() if key != shorter][0]
            # Find proportion of shorter seq positions aligned w/ longer
            aligned_cnt = 0  # Positions with content in short seq with content also in long seq
            for pos in range(full_aln.get_alignment_length()):
                if seqs[shorter].seq[pos] != '-' and seqs[longer].seq[pos] != '-':
                    aligned_cnt += 1
            aligned_proportion = aligned_cnt / len(str(seqs[shorter].seq).replace('-', ''))
            with pw_coverage_lock:
                pw_coverage_full[tuple(seqs.keys())] = aligned_proportion
        print(f"PW coverages done for seq {j}")

    future_store = [pw_coverage_executor.submit(pw_coverage, j) for j in range(len(full_aln) - 1)]
    concurrent.futures.wait(future_store)

    # TODO: Now in nested function for multi-threading - remove this if it works
    # # Calculate pairwise alignment coverage (as aligned proportion of shorter sequence) for all seqs in full_aln
    # for j in range(len(full_aln) - 1):
    #     for k in range(j + 1, len(full_aln)):
    #         seqs = {full_aln[j].name: full_aln[j], full_aln[k].name: full_aln[k]}
    #         # Determine shortest ungapped seq
    #         shorter = min(seqs.keys(), key=lambda seq: len(str(seqs[seq].seq).replace('-', '')))
    #         longer = [key for key in seqs.keys() if key != shorter][0]
    #         # Find proportion of shorter seq positions aligned w/ longer
    #         aligned_cnt = 0  # Positions with content in short seq with content also in long seq
    #         for pos in range(full_aln.get_alignment_length()):
    #             if seqs[shorter].seq[pos] != '-' and seqs[longer].seq[pos] != '-':
    #                 aligned_cnt += 1
    #         aligned_proportion = aligned_cnt / len(str(seqs[shorter].seq).replace('-', ''))
    #         pw_coverage_full[tuple(seqs.keys())] = aligned_proportion

    to_trim = []  # Indices of columns to trim
    under_co_cnt = []

    # Multi-thread column assessment for trimming
    col_assess_executor = concurrent.futures.ThreadPoolExecutor(max_workers=max_threads)
    to_trim_lock = threading.Lock()  # For appending cols to trim
    under_co_lock = threading.Lock()  # For keeping track of # of cols assessed for trimming

    # Method for assessing columns for potential trimming
    def assess_col(i):

        # Check if column occupancy is below base_threshold and assessable for trimming
        if 1 - (full_aln[:, i].count('-') / len(full_aln)) < base_threshold:
            with under_co_lock:
                under_co_cnt.append(i)

            # Collect seqs with content in column i
            sub_aln = [full_aln[j] for j in range(len(full_aln)) if full_aln[j][i] != '-']

            # Sort coverage comparisons by whether both seqs have content in column of interest
            pw_coverage_pos = {}  # Both seqs have content in column of interest
            pw_coverage_neg = {}  # One or both do not
            sub_pairs = []  # List of immutable sets representing pairs of seqs in sub_aln
            for j in range(len(sub_aln) - 1):
                for k in range(j, len(sub_aln)):
                    sub_pairs.append(tuple([sub_aln[j].name, sub_aln[k].name]))
            for seqs, coverage in pw_coverage_full.items():
                if seqs in sub_pairs:
                    pw_coverage_pos[seqs] = coverage
                else:
                    pw_coverage_neg[seqs] = coverage

            # Maintain column only if positive pairwise coverages are significantly higher than negatives
            t, p = stats.ttest_ind(list(pw_coverage_pos.values()), list(pw_coverage_neg.values()), equal_var=False,
                                   alternative='greater')
            if p >= p_threshold:  # Trim if NOT significantly higher coverage in aligned seqs
                with to_trim_lock:
                    to_trim.append(i)

            print(f'Assessed col {i}')

        else:
            print(f'Col {i} above CO threshold')

    future_store = [col_assess_executor.submit(assess_col, i) for i in range(full_aln.get_alignment_length())]
    concurrent.futures.wait(future_store)

    # TODO: Now in nested function for multi-threading - remove if all works
    # for i in range(full_aln.get_alignment_length()):   # For each column

        # # Check if column occupancy is below base_threshold and assessable for trimming
        # if 1 - (full_aln[:, i].count('-') / len(full_aln)) < base_threshold:
        #     under_co_cnt += 1
        #
        #
        #     # Collect seqs with content in column i
        #     sub_aln = [full_aln[j] for j in range(len(full_aln)) if full_aln[j][i] != '-']
        #
        #     # Sort coverage comparisons by whether both seqs have content in column of interest
        #     pw_coverage_pos = {}  # Both seqs have content in column of interest
        #     pw_coverage_neg = {}  # One or both do not
        #     sub_pairs = []  # List of immutable sets representing pairs of seqs in sub_aln
        #     for j in range(len(sub_aln)-1):
        #         for k in range(j, len(sub_aln)):
        #             # TODO: Check if below line is correct (was throwing error, have made a change but not sure its right)
        #             sub_pairs.append(tuple([sub_aln[j].name, sub_aln[k].name]))
        #     for seqs, coverage in pw_coverage_full.items():
        #         if seqs in sub_pairs:
        #             pw_coverage_pos[seqs] = coverage
        #         else:
        #             pw_coverage_neg[seqs] = coverage
        #
        #     # Maintain column only if positive pairwise coverages are significantly higher than negatives
        #     t, p = stats.ttest_ind(list(pw_coverage_pos.values()), list(pw_coverage_neg.values()), equal_var=False,
        #                            alternative='greater')
        #     if p >= p_threshold:  # Trim if NOT significantly higher coverage in aligned seqs
        #         to_trim.append(i)
        #
        #         trimmed_cnt += 1
        #
        #     print(f'Assessed col {i}')
        #
        # else:
        #     print(f'Col {i} above CO threshold')

    # Trim appropriate columns
    trimmed = trim_aln(full_aln, to_trim)

    print(f'Trimmed {len(to_trim)} of {len(under_co_cnt)} under occupancy threshold.')

    # Trimmed cols may be out of order due to multi-threading
    to_trim.sort()

    print(to_trim)

    # Write trimmed aln to file if specified, else return MSA object
    if out_file:
        AlignIO.write(trimmed, out_file, format)
#         return to_trim   # Return trimmed cols as list
        return to_trim
    else:
        return trimmed





In [61]:
infile = "/Users/sebs_mac/uni_OneDrive/honours/data/gb1/profile_creation/alns/gb1_30%_sim_nr90_high_acc.aln"
trimmed = smart_trim(in_file=infile, base_threshold=0.3) #, out_file="gb1_30%_sim_nr90_high_acc_trimmed_0.05.aln")

PW coverages done for seq 1PW coverages done for seq 3

PW coverages done for seq 0
PW coverages done for seq 2
PW coverages done for seq 7
PW coverages done for seq 4
PW coverages done for seq 8
PW coverages done for seq 5
PW coverages done for seq 9
PW coverages done for seq 6
PW coverages done for seq 10
PW coverages done for seq 11
PW coverages done for seq 12
PW coverages done for seq 13
PW coverages done for seq 15
PW coverages done for seq 14
PW coverages done for seq 16
PW coverages done for seq 17
PW coverages done for seq 18
PW coverages done for seq 19
PW coverages done for seq 20
PW coverages done for seq 21
PW coverages done for seq 22
PW coverages done for seq 23
PW coverages done for seq 24
PW coverages done for seq 25
PW coverages done for seq 26
PW coverages done for seq 27
PW coverages done for seq 28
PW coverages done for seq 29
PW coverages done for seq 30
PW coverages done for seq 31
PW coverages done for seq 32
PW coverages done for seq 33
PW coverages done for se

/var/folders/tt/p2_ty_j96x33hxxx4tfrnsq40000gn/T/ipykernel_2569/3395642634.py:109: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t, p = stats.ttest_ind(list(pw_coverage_pos.values()), list(pw_coverage_neg.values()), equal_var=False,
/Users/sebs_mac/miniconda3/envs/embed/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/sebs_mac/miniconda3/envs/embed/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


Assessed col 3Col 19 above CO threshold
Col 20 above CO threshold
Col 21 above CO threshold
Col 22 above CO threshold
Col 23 above CO threshold
Col 24 above CO threshold
Col 25 above CO threshold
Col 26 above CO threshold
Col 27 above CO threshold
Col 28 above CO threshold

Col 32 above CO threshold
Col 34 above CO threshold
Col 35 above CO threshold
Col 36 above CO threshold
Col 37 above CO threshold
Col 38 above CO threshold
Col 40 above CO threshold
Col 41 above CO threshold
Col 42 above CO threshold
Col 43 above CO threshold
Col 44 above CO threshold
Assessed col 4
Assessed col 29
Assessed col 31
Assessed col 30
Assessed col 45
Assessed col 46
Assessed col 47
Col 53 above CO threshold
Col 54 above CO threshold
Assessed col 39Assessed col 51
Col 56 above CO threshold

Col 58 above CO threshold
Assessed col 50
Assessed col 49
Assessed col 52
Col 62 above CO threshold
Col 63 above CO threshold
Assessed col 57
Assessed col 65
Assessed col 60
Assessed col 66
Col 68 above CO threshold
Co

In [92]:
orig_aln = st.read_aln_file("/Users/sebs_mac/uni_OneDrive/honours/data/gb1/profile_creation/alns/gb1_30%_sim_nr90_high_acc.aln")
orig_aln.head()

Reading the alignment: /Users/sebs_mac/uni_OneDrive/honours/data/gb1/profile_creation/alns/gb1_30%_sim_nr90_high_acc.aln
Checking for bad characters: ['B', 'J', 'X', 'Z', 'U']
Number of seqs: 335


,id,sequence
0,SPG1_STRSG/1-448,----MEKEKKVKYFLRKSAFGLASVSAAF---L-VG-ST-VFAVDS...
1,UniRef100_Q93EM8/1-351,----MEKEKKVKYFLRKSAFGLASVSAAF---L-VSGAL-ENTIT-...
2,UniRef100_A0A448DI38/1-351,----MEKEKKMKYFLRKSAFGLASVSAAF---L-VGTTL-DTTMT-...
3,UniRef100_UPI0011BD1DD8/3-449,-----KNNTNRHYSLRKLKTGTASVAVAL---T-LGAGF-ANQTE-...
4,UniRef100_A0A380JVQ2/1-402,----MEKEKKVKYFLRKSAFGLASVSAAF---L-VGVTT-VSAAD-...


In [97]:
wt = orig_aln["sequence"][0] # grab the wild type as well 
# columns that have been removed by smart_trim from the original alignment. 
trimmed = [0, 3, 39, 45, 46, 47, 52, 57, 64, 71, 72, 86, 87, 88, 89, 90, 98, 100, 101, 102, 104, 107, 108, 109, 110, 111, 112, 113, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 131, 132, 133, 134, 135, 136, 137, 138, 153, 155, 160, 161, 163, 164, 174, 179, 180, 181, 186, 187, 198, 199, 200, 242, 243, 276, 297, 298, 307, 308, 328, 329, 339, 357, 358, 365, 366, 372, 373, 374, 376, 384, 385, 386, 389, 396, 399, 404, 406, 414, 426, 427, 428, 430, 433, 434, 435, 436, 447, 449, 469, 525, 526, 528, 533, 552, 581, 582, 583, 596, 602, 603, 630, 631, 660, 669, 701, 712, 713, 723, 739, 754, 755, 756, 798, 799, 806, 810, 816, 817, 818, 821, 823, 831, 832, 833, 892, 915, 916, 917, 924, 932, 933, 934, 965, 1022, 1024, 1025, 1026, 1027, 1028, 1031, 1032, 1033, 1034, 1088, 1109, 1110, 1111, 1130, 1131, 1214, 1215, 1216, 1222, 1230, 1265, 1266, 1273, 1295, 1296, 1297, 1347, 1348, 1354, 1365, 1374, 1393, 1402, 1404, 1410, 1417, 1418, 1424, 1447, 1463, 1473, 1486]

In [93]:
# read in the variants
vars = pd.read_csv("/Users/sebs_mac/git_repos/dms_data/DMS_ProteinGym_substitutions/SPG1_STRSG_Wu_2016.csv")
vars.head()

,mutant,mutated_sequence,DMS_score,DMS_score_bin
0,D266A,MEKEKKVKYFLRKSAFGLASVSAAFLVGSTVFAVDSPIEDTPIIRN...,2.401243,1
1,D266A:G267A,MEKEKKVKYFLRKSAFGLASVSAAFLVGSTVFAVDSPIEDTPIIRN...,0.259549,1
2,D266A:G267A:V280A,MEKEKKVKYFLRKSAFGLASVSAAFLVGSTVFAVDSPIEDTPIIRN...,6.201135,1
3,D266A:G267A:V280C,MEKEKKVKYFLRKSAFGLASVSAAFLVGSTVFAVDSPIEDTPIIRN...,2.705083,1
4,D266A:G267A:V280E,MEKEKKVKYFLRKSAFGLASVSAAFLVGSTVFAVDSPIEDTPIIRN...,0.004726,0


In [94]:
# make into a numpy msa
vars.rename(columns={"mutant": "id", "mutated_sequence": "sequence"}, inplace=True)
vars_msa, _, _ = st.convert_msa_numpy_array(vars)

Sequence weight numpy array created with shape (num_seqs, columns):  (149360, 448)


In [99]:
# we need to resize our variants to match the WT in the original alignment 
resized_variants = np.zeros((vars_msa.shape[0], len(wt)))

# go through the wt, every time we see content, we know we can put the variant information 
# at this column as we're only making substituion mutations. 
var_position = 0 
for idx, residue in enumerate(wt):
    if residue != "-":
        resized_variants[:, idx] = vars_msa[:, var_position]
        var_position += 1

resized_variants.shape

(149360, 1576)

In [100]:
from evoVAE.utils.seq_tools import IDX_TO_AA

# save the aligned variants before trimming 
seqs = []
ids = []
for i in range(resized_variants.shape[0]): 
    seq = "".join([IDX_TO_AA[x] for x in resized_variants[i, :]])
    seqs.append(seq)
    ids.append(vars["id"][i])

var_fasta = pd.DataFrame({"id": ids, "sequence": seqs})
st.write_fasta_file("gb1_variants.aln", var_fasta)

In [107]:

# now trim the variants to match the alignment produced by smart_trim
non_trimmed_indices = [x for x in range(resized_variants.shape[1]) if x not in trimmed]
trimmed_variants = resized_variants[:, non_trimmed_indices]

seqs = []
ids = []
for i in range(trimmed_variants.shape[0]): 
    seq = "".join([IDX_TO_AA[x] for x in trimmed_variants[i, :]])
    seqs.append(seq)
    ids.append(vars["id"][i])

var_fasta = pd.DataFrame({"id": ids, "sequence": seqs})
st.write_fasta_file("gb1_variants_trimmed.aln", var_fasta)